In [13]:
import pandas as pd

# List of CSV files
csv_files = ["eeg_left.csv", "eeg_right.csv", "eeg_top.csv"]

# List of corresponding meta values
direction_values = ["left", "right", "top"]

# Create an empty DataFrame
result_df = pd.DataFrame()

# Iterate through CSV files and meta values
for csv_file, direction_value in zip(csv_files, direction_values):
    # Read CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    # Add a new column 'direction' with the corresponding value
    df["direction"] = direction_value

    # Concatenate the current DataFrame with the result DataFrame
    result_df = pd.concat([result_df, df], ignore_index=True)

# Display the resulting DataFrame
print(result_df)

         timestamps     TP9      AF7      AF8     TP10  Right AUX direction
0      1.700338e+09 -10.254   28.809   21.484   -9.277        0.0      left
1      1.700338e+09 -83.984 -458.496 -333.984 -126.953        0.0      left
2      1.700338e+09 -25.391 -720.703 -399.414  -46.387        0.0      left
3      1.700338e+09  52.246 -321.777 -200.195   76.660        0.0      left
4      1.700338e+09  11.719 -211.914 -211.426   23.438        0.0      left
...             ...     ...      ...      ...      ...        ...       ...
88627  1.700338e+09   0.000 -397.461    0.000  -85.449        0.0       top
88628  1.700338e+09   0.000 -798.340    0.000  -81.543        0.0       top
88629  1.700338e+09   0.000    7.324    0.000    4.883        0.0       top
88630  1.700338e+09   0.000  420.898    0.000   24.902        0.0       top
88631  1.700338e+09   0.000 -185.547    0.000  -62.988        0.0       top

[88632 rows x 7 columns]


In [ ]:
import pandas as pd
import numpy as np
from tslearn.preprocessing import TimeSeriesScalerMinMax
from tslearn.utils import to_time_series_dataset
from tslearn.shapelets import ShapeletModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a DataFrame named df with columns ["direction", "TP9", "AF7", "AF8", "TP10"]
# Parameters
epoch_length = 50

# Function to preprocess the data and create time series epochs
def preprocess_data(df, epoch_length):
    time_series_fragments = []
    labels = []

    for direction in df["direction"].unique():
        eeg_data_direction = df[df["direction"] == direction][["TP9", "AF7", "AF8", "TP10"]].values
        total_epochs = len(eeg_data_direction) // epoch_length

        eeg_epochs = [eeg_data_direction[i * epoch_length: (i + 1) * epoch_length] for i in range(total_epochs)]

        time_series_fragments.extend(eeg_epochs)
        labels.extend([direction] * total_epochs)

    # Convert the list of epochs to a time series dataset
    time_series_dataset = to_time_series_dataset(time_series_fragments)

    # If needed, you can also normalize the time series data
    scaler = TimeSeriesScalerMinMax()
    time_series_dataset_scaled = scaler.fit_transform(time_series_dataset)

    return time_series_dataset_scaled, np.array(labels)


In [ ]:
# Preprocess the data
X, y = preprocess_data(df, epoch_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Build a simple Shapelet model (you can experiment with other classifiers in tslearn)
model = ShapeletModel(n_shapelets_per_size={5: 10}, optimizer="sgd", weight_regularizer=.01, max_iter=100)

# Fit the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)